# Estimating playing strength in chess
This analysis aims to produce a model that is capable of predicting a player's strength in terms of rating, based on the information from a game.

## About the dataset
The data for this project is from a collection of games played on one of the largest chess sites lichess.org.<br><br>

The dataset consists of games from a large online tournament, the yearly classical arena (Y4t9Lk9R), which has about 16,000 games. The data can be downloaded via the following link.
> https://lichess.org/api/tournament/Y4t9Lk9R/games?evals=true


## Assumptions about playing strength
A frequently used statistic for estimating playing strength is average centipawn loss (aCPL). This will be one of the main variables that we will compare to the players' actual ratings. In addition, the following variables are thought to be of relevance for this analysis:
- Rating difference between players
- Number of moves played
- Number of blunders (big mistakes)
- Blunder rate (number of blunders / number of moves)

## 1. Import packages

In [ ]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import seaborn as sns

## 2. Data collection

In [ ]:
arena_pgn="lichess_tournament_2020.05.15_Y4t9Lk9R_yearly-classical.pgn"

# These are the tags we want to extract from the pgn and use as columns in the dataframe
# For the tournament data, the timecontrol is not interesting, as all games are played with the same time control
# but for general purposes, the tag may be relevant.
cols=["WhiteElo","BlackElo","TimeControl","ECO","Moves"]

In [ ]:
# This function separates tags/moves into pairs of keys and values
# The function has been tested and validated

def categorize_lines(line):
    header=re.search(r"\[(\w+)",line)
    if header is None:
        key="Moves"
        val=line
    else:
        key=header[1]
        val=re.search(r"\"(.+)\"",line)[1]
    return(key,val)

In [ ]:
# Collect all info into a list of dicts
# Start with a clean list and a clean game dict
my_list=[]
game={}
key=""
with open(arena_pgn) as pgn:
    for line in pgn:
        if key=="Moves":
            my_list.append(game)
            key=""
            game={}
        entry=line.strip()
        if entry!="":
            key,val=categorize_lines(entry)
            # So far, this is what has been tested above
            if key in cols:
                game[key]=val

In [ ]:
df=pd.DataFrame(my_list,columns=cols)
df.drop(0,inplace=True)

In [ ]:
# Filter the rows so we only get annotated games
df["Eval"]=df["Moves"].apply(lambda x: "eval" in x)
df=df[df["Eval"]]

In [ ]:
# Ensure that the rating values are entered as numbers
df[["WhiteElo","BlackElo"]]=df[["WhiteElo","BlackElo"]].apply(pd.to_numeric)

In [ ]:
df.head()

Based on the output above, it seems that the conversion from pgn to dataframe has worked well.

## 3. Data wrangling
So far, we have only extracted the data in "raw" form from the pgn file. A few more columns are needed before the analysis can be performed.

In [ ]:
# Function for calculating aCPL per player
def get_aCPL(moves,start=0):
    # Find all scores
    scores=np.array(re.findall(r"eval (-?[0-9]+\.[0-9]+)",str(moves)),dtype=float)
    # Calculate score difference, move by move
    CPL=abs(np.diff(scores))
    # Calculate average score by color
    aCPL_w=CPL[(start+1)::2].mean()*100
    aCPL_b=CPL[start::2].mean()*100
    num_moves=int(len(CPL)/2)
    return(aCPL_w,aCPL_b,num_moves)

In [ ]:
def get_blunders(moves,threshold):
    # Find all scores
    scores=np.array(re.findall(r"eval (-?[0-9]+\.[0-9]+)",str(moves)),dtype=float)
    # Calculate score difference, move by move
    CPL=abs(np.diff(scores))
    CPL_w=CPL[1::2]
    CPL_b=CPL[::2]
    blunders_w=len(CPL_w[CPL_w>threshold])
    blunders_b=len(CPL_b[CPL_b>threshold])
    return(blunders_w,blunders_b)

In [ ]:
# Calculate CPL per player and the number of moves per game
df["white_cpl"]=df["Moves"].apply(lambda x: get_aCPL(x)[0])
df["black_cpl"]=df["Moves"].apply(lambda x: get_aCPL(x)[1])
df["num_moves"]=df["Moves"].apply(lambda x: get_aCPL(x)[2])

In [ ]:
# We create categories of players, by rating
player_groups=["noob","patzer","grandpatzer","master","grandmaster"]
rtg_bins = [500,1400,1800,2200,2500,2700]

# Then we add the category names to the dataframe
df["category_w"]=pd.cut(df['WhiteElo'].astype("int",errors="ignore"), rtg_bins, labels=player_groups, include_lowest=True)
df["category_b"]=pd.cut(df['BlackElo'].astype("int",errors="ignore"), rtg_bins, labels=player_groups, include_lowest=True)

In [ ]:
# Calculate the rating difference between players
df["Rating_diff"]=df["BlackElo"]-df["WhiteElo"]

In [ ]:
# Calculate the number of blunders per player
# The threshold value here is 1.5 pawns (=1500 CP)
df["blunders_w"]=df["Moves"].apply(lambda x: get_blunders(x,1.5)[0])
df["blunders_b"]=df["Moves"].apply(lambda x: get_blunders(x,1.5)[1])

In [ ]:
# Calculate the blunder rate
df["blunder_rate_w"]=df["blunders_w"]/df["num_moves"]
df["blunder_rate_b"]=df["blunders_b"]/df["num_moves"]

In [ ]:
df.info()

The table above shows that all columns are complete, i.e. there are no non-null values (NaNs) in the dataset. Also, all the columns have appropriate datatypes.

## 4. Data exploration

In [ ]:
# For starters, let's take a look at the descriptive statistics for the variables
df.describe()

From the table above, we can make a number of observations:
- The average rating is about 1770
- The average CPL is just below 120, which is surprisingly high
- The median CPL is considerably lower than the average, indicating an asymmetrical distribution
- On average, a game lasts about 28 moves
- On average, a player blunders 4 times during a game, or about 15% of the moves
- The stats are almost identical for white and black

In [ ]:
# Let's take a look at the distribution of players
pd.DataFrame({"White": df["category_w"].value_counts(),"Black":df["category_b"].value_counts()})

We see that the players in the patzer and grandpatzer categories account for the vast  majority of players. The result of the analysis will therefore be most applicable for these groups. Also, there is only one player in the grandmaster group. Any conclusions from this data set regarding that group will therefore not be relevant.

In [ ]:
df[["WhiteElo","BlackElo"]].plot.hist(bins=100, alpha=0.7)

Once again, we see that the rating distributions are almost identical for white and black players. The spike at 1500 is probably because the starting rating for new accounts is 1500. The histograms are slightly skewed to the right.

In [ ]:
# Let's take a look at the players with a rating of exactly 1500.
# Since the distributions are similar for both colors, we will use white as an example.
df[df["WhiteElo"]==1500].describe()

The table above shows descriptive statistics for (white) players with a rating of *exactly* 1500. The distributions are approximately the same as for all players, indicating that the rating is not representative of the rating group overall. They will therefore be excluded from the dataset. 

In [ ]:
# We remove players with a rating of exactly 1500, so that they will not "contaminate" the results.
df=df[df["WhiteElo"]!=1500]
df=df[df["BlackElo"]!=1500]

In [ ]:
# After filtering out the 1500-players, the rating distribution looks like this (white only)
df.groupby("category_w")["WhiteElo"].plot.hist(bins=100)
plt.legend()
plt.title("Histogram of player ratings")
plt.xlabel("Player rating (white)")
plt.ylabel("Frequency")

In [ ]:
# Let's take a look at the correlations between the main variables
# pd.plotting.scatter_matrix(df[["WhiteElo","BlackElo","white_cpl","black_cpl","blunder_rate_w","blunder_rate_b"]],figsize=(15,10))
# plt.suptitle("Scatter matrix",fontsize=15)
# Has been removed to save space.

In [ ]:
# We can also generate a correlation matrix
print("Correlation matrix")
df[["WhiteElo","BlackElo","white_cpl","black_cpl","blunder_rate_w","blunder_rate_b"]].corr()

A few observations from the matrix plot and correlation matrix:
- There seems to be a correlation between the players' ratings. This suggests that players tend to be paired with players around their rating level.
- CPL and rating seems to be correlated, but it is not very strong
- Blunder rate and CPL have a quite strong correlation, but this is not so surprising, as the blunder rate is derived from the CPL scores
- Interestingly, there seems to be a relatively strong correlation between CPL scores between players. This may be a result of the pairings, and that players of the same rating tend to have equal CPL scores


In [ ]:
# Since the centipawn loss is the variable we're most interested in, let's take a closer look at the distribution of the variable.
df[["white_cpl","black_cpl"]].plot.hist(bins=100, alpha=0.7)
plt.title("Histogram of average centipawn loss")
plt.xlabel("Average centipawn loss (aCPL)")

The CPL scores resembles an exponential distribution, with a peak around 30 CPL and a long tail. The histograms are close to zero from around 600 CPL. This indicates that linear regression can be problematic, and that transformation may be required.

In [ ]:
# The quadratic root is often a good transformation of exponentially distributed data
df["cpl_trans"]=df["white_cpl"].apply(lambda x: x**(1/4))
plt.hist(df["cpl_trans"],bins=100)
plt.title("Transformed CPL")

Transforming the data with the quadratic root makes the distribution relatively symmetrical. This may facilitate the further analysis.

In [ ]:
# After the transformation, the regression plot of rating versus CPL (transformed) looks like this:
sns.regplot(y=df["WhiteElo"],x=df["cpl_trans"])
plt.title("Regression plot")
plt.xlabel("CPL (transformed)")
plt.ylabel("Rating (white)")

This scatter plot gives a much clearer indication of the correlation between CPL and rating. However, the regression line seems to be a bit flat. One would expect it to have a steeper slope and an intercept at about 2500. The orientation of the plot suggests that this could be a correct estimate, but that outliers influence the result.

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(15,7))
df.groupby("category_w")["white_cpl"].plot.hist(bins=100,alpha=0.6,ax=ax[0])
df.groupby("category_w")["cpl_trans"].plot.hist(bins=100,alpha=0.6,ax=ax[1])
ax[0].legend()
plt.suptitle("Histogram of CPL by player group",fontsize=15)
ax[0].set_xlabel("CPL")
ax[1].set_xlabel("CPL (transformed")

In [ ]:
# A slightly more simple plot is a bar plot of just the average values for each player group.
fig,ax=plt.subplots(1,2,figsize=(15,7))
plt.suptitle("CPL by player group",fontsize=15)
df.groupby("category_w")["white_cpl"].mean().plot.barh(ax=ax[0])
df.groupby("category_w")["cpl_trans"].mean().plot.barh(ax=ax[1])
ax[0].set_xlabel("CPL")
ax[1].set_xlabel("CPL (transformed)")
ax[0].set_ylabel("Player group")
ax[1].set_ylabel("Player group")



In [ ]:
# We create a similar plot for blunder rate
df.groupby("category_w")["blunder_rate_w"].mean().plot.barh()
plt.xlabel("Average blunder rate\n(threshold=1.5)")
plt.title("Blunder rate per player group",fontsize=15)
plt.ylabel("Player group")

From these plots, we can conclude that our a priori assumptions about the relationships between player rating and the variables centipawn loss (CPL) and blunder rate, respectively, seems to hold fairly well at an overall level. Observe that the grandmaster category only consists of a single player, so the values are not relevant.

In [ ]:
df.to_csv("Lichess classical arena.csv",index=False)

In [ ]:
fig=plt.figure()
ax=fig.add_subplot()
sns.kdeplot(df["WhiteElo"],df["white_cpl"],cmap="Reds",shade=True,ax=ax,alpha=0.7)
sns.kdeplot(df["BlackElo"],df["black_cpl"],cmap="Blues",shade=True,ax=ax,alpha=0.3)
ax.plot([750,2500],[140,10],"g--")
ax.set_ylim(0,250)
plt.title("Contour plot of player rating vs CPL",fontsize=15)
plt.xlabel("Rating")
plt.ylabel("CPL")

This plot indicates that there is a clear relationship between rating and CPL. However, the amount of variation is huge. Just as an example, a CPL value of 50 gives a rating range of approximately 1400 to 2300.

In [ ]:
fig=plt.figure()
ax=fig.add_subplot()
sns.kdeplot(df["WhiteElo"],df["blunder_rate_w"],cmap="Reds",shade=True,ax=ax,alpha=0.7)
sns.kdeplot(df["BlackElo"],df["blunder_rate_b"],cmap="Blues",shade=True,ax=ax,alpha=0.3)
ax.plot([750,2250],[0.4,0.01],"g--")
ax.set_ylim(0,0.6)
plt.title("Contour plot of player rating vs blunder rate",fontsize=15)
plt.xlabel("Rating")
plt.ylabel("Blunder rate")

## 5. Building a model
Several attempts were made in this phase, and several models were discarded.
- WhiteElo ~ white_cpl gives R2=0.082
- WhiteElo ~ white_cpl + Rating_diff gives R2=.302. Quite good, but intercept=1854 - unreasonable
- WhiteElo ~ white_cpl + Rating_diff + num_moves gives r2=0.371 and intercept=1675. num_moves has param 6.5
- WhiteElo ~ white_cpl + Rating_diff + num_moves*blunders_w gives R2=0.410. Best R2 so far, but high condition no.
- WhiteElo ~ white_cpl + Rating_diff + num_moves + blunder_rate_w gives R2=0.418, but a high condition no.
- WhiteElo ~ white_cpl + Rating_diff + num_moves + blunders_w gives R2=0.401 and an acceptable condition no.
- The transformed CPL values gave only a slightly better result, so for easier interpretation, the original CPL values are used.

In [ ]:
model=smf.ols(formula="WhiteElo ~ white_cpl + Rating_diff + num_moves + blunders_w", data=df)
result=model.fit()
result.summary()

In [ ]:
plt.hist(result.resid,bins=100)
plt.title("Histogram of residuals")

The residuals are quite large, indicating that the predictive value of the model is of little practical relevance.

### Using the full dataset
So far, the analysis has been performed on just one of the players (white). We will therefore take another step and include both colors in the analysis.

In [ ]:
df.info()

In [ ]:
# Note that the rating difference is calculated from White's point of view, so the values have to be reversed
white=pd.DataFrame({"Rating":df["WhiteElo"],"CPL":df["white_cpl"],"RatingDiff":df["Rating_diff"],"nmoves":df["num_moves"],"nblunders": df["blunders_w"],"category":df["category_w"]})
black=pd.DataFrame({"Rating":df["BlackElo"],"CPL":df["black_cpl"],"RatingDiff":-df["Rating_diff"],"nmoves":df["num_moves"],"nblunders": df["blunders_b"],"category":df["category_b"]})
df2=pd.concat([white,black])

In [ ]:
df2.to_csv("Lichess stacked data.csv",index=False)

In [ ]:
df2.head()

In [ ]:
df2.info()

In [ ]:
# Let's try the same simple approach as above, plotting Rating vs CPL.
ax=sns.kdeplot(df2["Rating"],df2["CPL"],cmap="Blues",shade=True)
ax.plot([750,2500],[140,10],"g--")
ax.set_ylim(0,250)

In [ ]:
slope=(2500-750)/130
round(slope,2)

It seems that this crude model holds quite well for the complete dataset. There is a clear orientation to the plot, but there is also a great deal of uncontrolled variation. According to this model, each CPL is worth about 13 rating points.

In [ ]:
# Testing the regression model with the full dataset
full_model=smf.ols("Rating ~ CPL + RatingDiff + nmoves + nblunders",data=df2)
results2=full_model.fit()
results2.summary()

The R2 value drops slightly from the initial model (0.40 to 0.39), but the results are the same overall. All parameters are statistically significant, and we arrive at the following formula:
<br><br>
$Rating = 1655 - 0.20*CPL -0.45*RatingDiff + 8.55*nmoves -22*nblunders$
<br><br>
The parameter for CPL suggests that each CPL is worth only 0.2 rating points, and that the number of blunders is the main predictor with 22 rating points per blunder (threshold 1.5) along with the number of moves. 

In [ ]:
# Calculate the model prediction
df2["Rating_pred"]=results2.predict(df2[["CPL","RatingDiff","nmoves","nblunders"]])

In [ ]:
# We can evaluate the results by plotting the predicted values against the actual ratings
sns.regplot(x=df2["Rating_pred"],y=df2["Rating"])

We can see that the correlation between predicted and actual values is ok. The regression line gives approximately the same results for both values. However, the main problem is that there is a lot of unexplained variation in the dataset. This reduces the predictive value of the model.

In [ ]:
plt.hist(results2.resid,bins=100)
plt.title("Histogram of residuals\nModel 2")

The residuals resembles a standard distribution. The average seems to be around zero, and the standard deviation approximately 200. This means that we generally have a prediction error of $\pm$ 400 rating points. This is quite a lot!

In [ ]:
fig=plt.figure(figsize=(15,6))
fig.suptitle("Residual plots vs observed and predicted rating",fontsize=15)
ax1=fig.add_subplot(121)
ax2=fig.add_subplot(122)
ax1.scatter(y=results2.resid,x=df2["Rating"])
ax1.set_xlabel("Rating")
ax2.scatter(y=results2.resid,x=df2["Rating_pred"])
ax2.set_xlabel("Predicted rating")
ax1.set_ylabel("Residual")

There is no correlation between the residuals and the predicted values, but the residuals seem to increase with the actual rating values. This is consistent with our initial observations of the scatterplot of CPL vs Rating, where the variation was very high at low CPL levels.

## Applying the model to estimate performance
As indicated above, the precision of this model is simply not good enough for accurately predicting playing strength from a single game. However, as with many things in life, consistency is a key factor. Therefore, it will be more relevant to test the model on several games from a single player.<br><br>
Being a chess player myself, I don't mind being a guinea pig. I have therefore downloaded a list of annotated games from my own profile on lichess.

In [ ]:
# My personal games
my_pgn="lichess_Nietsoj_2020-05-19.pgn"

In [ ]:
# Since the analysis will focus on my own games, we need to add player names for each game. Since I have played all of them, one color is enough. 
cols.append("White")

In [ ]:
my_list=[]
game={}
key=""
with open(my_pgn) as pgn:
    for line in pgn:
        if key=="Moves":
            my_list.append(game)
            key=""
            game={}
        entry=line.strip()
        if entry!="":
            key,val=categorize_lines(entry)
            if key in cols:
                game[key]=val

In [ ]:
df3=pd.DataFrame(my_list,columns=cols)
df3.drop(0,inplace=True)

The data has been collected from the pgn file, but there are some adjustments that need to be made before moving forward. Lichess has different ratings for different time controls, so we need to identify the slower time controls, i.e. 15 minutes and more.


In [ ]:

df3["TimeControl"].value_counts()

In [ ]:
# Identify the initial time for each game
df3["TimeControl"]=df3["TimeControl"].apply(lambda x: int(x[:x.find("+")]))

In [ ]:
# We are only interested in timecontrols above 15 minutes, i.e. 900 seconds.
df3=df3[df3["TimeControl"]>=900]

In [ ]:
# Ensure that the rating values are entered as numbers
df3[["WhiteElo","BlackElo"]]=df3[["WhiteElo","BlackElo"]].apply(pd.to_numeric)

In [ ]:
# Get the CPL scores and number of moves
df3["white_cpl"]=df3["Moves"].apply(lambda x: get_aCPL(x)[0])
df3["black_cpl"]=df3["Moves"].apply(lambda x: get_aCPL(x)[1])
df3["num_moves"]=df3["Moves"].apply(lambda x: get_aCPL(x)[2])

In [ ]:
# Calculate the rating difference between players
df3["Rating_diff"]=df3["BlackElo"]-df3["WhiteElo"]

In [ ]:
# Calculate the number of blunders per player
df3["blunders_w"]=df3["Moves"].apply(lambda x: get_blunders(x,1.5)[0])
df3["blunders_b"]=df3["Moves"].apply(lambda x: get_blunders(x,1.5)[1])

In [ ]:
df3.head()

In [ ]:
df3.info()

The final dataset consists of 33 games. All the data has now been gathered, but we need one more step before we can perform the test. We need to separate the games I've played with white and black.

In [ ]:
my_white=df3[df3["White"]=="Nietsoj"]
my_black=df3[df3["White"]!="Nietsoj"]

In [ ]:
white=pd.DataFrame({"Rating":my_white["WhiteElo"],"CPL":my_white["white_cpl"],"RatingDiff":my_white["Rating_diff"],"nmoves":my_white["num_moves"],"nblunders": my_white["blunders_w"]})
black=pd.DataFrame({"Rating":my_black["BlackElo"],"CPL":my_black["black_cpl"],"RatingDiff":-my_black["Rating_diff"],"nmoves":my_black["num_moves"],"nblunders": my_black["blunders_b"]})
df4=pd.concat([white,black])

In [ ]:
df4.head()

In [ ]:
df4["Rating_pred"]=1655-0.2*df4["CPL"]-0.45*df4["RatingDiff"]+8.55*df4["nmoves"]-21.89*df4["nblunders"]

In [ ]:
df4["pred_crude"]=2635-df4["CPL"]*slope

In [ ]:
df4.describe()

In [ ]:
df4[["Rating","Rating_pred","pred_crude"]].plot(kind="box",showfliers=False,showmeans=True)
plt.title("Observed and predicted ratings\nMy own games",fontsize=13)
plt.xticks([1,2,3],["Observed","Regression\nestimate","Visual\nestimate"])
plt.ylabel("Rating")
plt.savefig("Boxplot_my_rating.png")
